In [127]:
#importing modules
import tweepy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re


from langdetect import detect_langs
from nltk.tokenize import word_tokenize
import nltk


[nltk_data] Error loading puntk: Package 'puntk' not found in index


False

In [ ]:
#twitter api keys
auth = tweepy.OAuth1UserHandler(
   'ObDcu9DovwyYkx5w5tJouFGvP', '6A2UhUI9YpVis96xea1xmZqR5b2JIMIpUAOQIjJBaNPkINwnOj', '459226572-g5WB6p6QAcVciJ50uN6SLTRVfZKDqS0Bcp4bFGqd', 'jzFTq24nII5LUsG0FjsSCsrS0GLWN5ZK5BPIMfiJceUls')

api = tweepy.API(auth, wait_on_rate_limit=True)


In [ ]:
search_words = "#wakandaforever OR #WakandaForever OR #blackpanther OR #WakandaForeverID OR #BlackPantherWakandaForever OR #BlackPanther OR #BlackPanther2"
search_query = search_words + " -filter:retweets AND -filter:replies"

In [ ]:
limit = 100000
tweets = tweepy.Cursor(api.search_tweets, q=search_query, tweet_mode='extended').items(limit)

In [ ]:
data=[]
for tweet in tweets:
    data.append([tweet.id,tweet.user.screen_name, tweet.created_at, tweet.full_text,tweet.user.location, tweet.retweet_count,tweet.favorite_count, tweet.user.followers_count, tweet.user.friends_count, tweet.user.verified, tweet.source])

In [ ]:
columns =['id','username','time_of_tweet', 'tweet', 'location', 'retweets', 'likes', 'followers', 'following', 'verified', 'tweet_source']
bp_tweets=pd.DataFrame(data, columns=columns)
bp_tweets.to_csv("BlackPanther-24-11.csv")

In [2]:
tweets = pd.read_csv('BlackPanther-24-11.csv')
tweets.sample(10)

,Unnamed: 0,id,username,time_of_tweet,tweet,location,retweets,likes,followers,following,verified,tweet_source
30098,30098,1592942899159011328,GeekWatchOne,2022-11-16 18:09:17+00:00,"Of course, we're reviewing #BlackPanther #Waka...",Ohio,1,2,1653,1051,False,Twitter Web App
32801,32801,1592770765346439168,tweetticator,2022-11-16 06:45:17+00:00,Black women rulers in the history of the Yorub...,NaN,0,2,626,89,False,Twitter Web App
34238,34238,1592709251797028869,BTO353,2022-11-16 02:40:51+00:00,Stan lee the creator of #Blackpanther https://...,NaN,0,0,67,329,False,Twitter for iPhone
23556,23556,1593353475483701248,Josh_Johnson79,2022-11-17 21:20:46+00:00,NAMOR AINT PLAYIN #BlackPanther,"Irmo,SC",0,0,446,822,False,Twitter for iPhone
29533,29533,1592970801271214080,Disney_Insiders,2022-11-16 20:00:09+00:00,Return to Wakanda.\n\nMarvel Studios’ Black Pa...,NaN,56,785,76184,43,True,Sprinklr
28249,28249,1593028249076256770,Wheeljack83,2022-11-16 23:48:26+00:00,Delicious refreshments and pastries at today’s...,"Los Angeles, CA",0,2,471,323,False,Twitter for iPhone
16190,16190,1594008040071745537,david_jgpeoples,2022-11-19 16:41:46+00:00,I didn’t expect many surprises in #WakandaFore...,NaN,0,2,71,43,False,Twitter for iPhone
5609,5609,1594874420694245376,TheGoldinChyld,2022-11-22 02:04:27+00:00,#BlackPanther\n#TChalla\n#WakandaForever https...,NaN,0,0,179,809,False,Twitter for Android
8768,8768,1594520818817667077,kathia_woods,2022-11-21 02:39:22+00:00,No petition or hate shall keep us from the mon...,"Philadelphia, PA",2,14,4803,3884,False,Twitter for iPhone
27904,27904,1593046525516865536,gnurpiyani,2022-11-17 01:01:03+00:00,"Saya telah mengirimkan #ShopeePayTHR 🧧. Ayo, a...",NaN,0,0,618,398,False,Twitter for iPhone


In [ ]:
tweets.info()

In [ ]:
tweets.head()

In [15]:
tweets.drop(columns='Unnamed: 0', inplace = True)

In [17]:
tweets['location'].fillna('None', inplace=True)

In [21]:
tweets['time_of_tweet'] = tweets['time_of_tweet'].astype('datetime64')

In [181]:
def get_hash(tweet):
    tweet = re.findall(r'\#\w+', tweet)
    return " ".join(tweet)

In [182]:
tweets['hashtags'] = tweets['tweet'].apply(get_hash)


In [185]:
hashtags = pd.Series(" ".join(tweets['hashtags']).split()).value_counts()
hashtags[:10]

#WakandaForever                20990
#BlackPanther                   9316
#BlackPantherWakandaForever     5887
#wakandaforever                 1997
#Namor                          1746
#blackpanther                   1297
#Marvel                         1211
#BlackPanther2                   998
#WakandaForeverID                978
#MarvelStudios                   808
dtype: int64

In [10]:
def get_languages(tweet):
    try:
        tweet = detect_langs(tweet)
    except: 
        tweet = "nolang"
    tweet = str(tweet).split(':')[0][1:]
    return tweet

In [11]:
tweets['language'] = tweets['tweet'].apply(get_languages)

In [14]:
tweets.language.value_counts()[:10]

en    24188
es     3334
da     1493
pt     1456
fr      776
id      512
ja      473
th      469
no      377
it      330
Name: language, dtype: int64

In [71]:
def get_links(tweets):
    link = re.findall(r'((http|ftp|https):\/\/)(([\w.-]*)\.([\w]*))', tweets)
    if len(link) < 1:
        return 'No'
    else: return 'Yes'


In [72]:
tweets['contains_media'] = tweets['tweet'].apply(get_links)

In [73]:
tweets.contains_media.value_counts()

In [167]:
cast = ['namor', 'shuri', 'ironheart', 'ramonda', 'namora', 
        'okoye', 'aneka', 'nakia', 'mbaku', 'attuma', 'everett', ]


def get_cast(tweets):
    tweets = tweets.lower()
    tokens = word_tokenize(tweets)
    cast_names = [name for name in tokens if name in cast]
    cast_names_title = [name.title() for name in cast_names]
    return " ".join(cast_names_title)

In [168]:
tweets['cast_name'] = tweets['tweet'].apply(get_cast)

In [177]:
names = pd.Series(" ".join(tweets['cast_name']).split()).value_counts()
# names.to_csv('names_freq.csv') ##save the generated dataframe
names

Namor        3953
Shuri        1985
Okoye         348
Namora        295
Ironheart     288
Nakia         220
Ramonda       216
Mbaku         202
Attuma        123
Aneka          53
Everett         6
dtype: int64

Happy


In [183]:
tweets.sample(10)

,id,username,time_of_tweet,tweet,location,retweets,likes,followers,following,verified,tweet_source,language,links,contains_media,cast_name,hashtags
23998,1593322801380851716,AndreeOrtega5,2022-11-17 19:18:53,De septimo arte 🎬🎬🎬📽🎞🎥📼📸📷 #WakandaForeverID #W...,"Chone, Ecuador",0,0,323,975,False,Twitter for Android,es,"[(https://, https, t.co, t, co)]",Yes,,#WakandaForeverID #WakandaPorSiempre #supercine
1542,1595495246900563968,djshortonesk,2022-11-23 19:11:24,Finally! #blackpanther #WakandaForever #regal ...,SC,0,0,1393,4988,False,Foursquare Swarm,en,"[(https://, https, t.co, t, co), (https://, ht...",Yes,,#blackpanther #WakandaForever #regal
17345,1593903062011084800,barnsgf,2022-11-19 09:44:37,spoiler #BlackPanther \n.\n.\n.\n.\n.\n.\n.\n....,midgard,0,4,264,287,False,Twitter for iPhone,da,"[(https://, https, t.co, t, co)]",Yes,,#BlackPanther
8579,1594537261613592576,guillegoony,2022-11-21 03:44:42,lo primero que pense al ver #WakandaForever fu...,Source,0,0,290,307,False,Twitter for Android,es,[],No,,#WakandaForever
12605,1594231777429635072,ey_jey_reyes,2022-11-20 07:30:49,Ang ganda ng soundtrack ng #BlackPantherWakand...,Spec in the Universe,0,1,609,408,False,Twitter for Android,tl,[],No,,#BlackPantherWakandaForever
3228,1595236224058408960,Ernesto_galv,2022-11-23 02:02:08,"Just watched #WakandaForever , on one hand sla...","Austin, TX",0,2,26,28,False,Twitter for iPhone,en,"[(https://, https, t.co, t, co)]",Yes,,#WakandaForever
10801,1594395883210170368,samueldcarrico,2022-11-20 18:22:55,Watch our review of #wakandaforever here: http...,"Tulsa, OK",0,0,463,268,False,Twitter for iPhone,en,"[(https://, https, t.co, t, co), (https://, ht...",Yes,,#wakandaforever
12804,1594211260488450048,morely542,2022-11-20 06:09:18,Si tienen la oportunidad de ver #WakandaForeve...,Bogota - Colombia,0,0,184,955,False,Twitter for iPhone,no,[],No,,#WakandaForever
11825,1594330040660959233,TEEMPTraining,2022-11-20 14:01:17,...The fierceness and strength of @ImAngelaBas...,DC/MD/VA with more reach,0,0,425,1514,False,Twitter Web App,en,[],No,,#WakandaForever #BlackPanther
19862,1593688842602655745,KiKiumai,2022-11-18 19:33:24,Went to watch #BlackPanther and I cried my eye...,"London, England",0,5,984,619,False,Twitter for iPhone,en,"[(https://, https, t.co, t, co)]",Yes,,#BlackPanther


In [ ]:
cv = tweets['location'].replace(np.nan, "None")

In [ ]:
cv[cv.str.contains('IL')][:60]
